Darcy-Weisbach equation  
$$
H_L=f\left(\frac{L}{D}\right)\left(\frac{v^2}{2g}\right)
$$
Colebrook White equation  
$$
\frac{1}{\sqrt{f}}=-2log{\left(\frac{k}{3.7D}+\frac{2.51}{R_e{f^{0.5}}}\right)}
$$
Swamee Jain friction factor approximation  
$$
f=\frac{0.25}{\left(log\left(\frac{k}{3.71D}+\frac{5.74}{R_e^{0.9}}\right)\right)^2}
$$

In [7]:
# Colebrook White headloss calculator

# Input
Q = 100 # l/s
D = 150 # mm Pipe diameter
v = 1*10**-6 # m**2/s Viscosity
g = 9.81 # m/s**2
L = 2500 # m length of pipe
k = 0.3 # mm surface roughness

# Calculations for first approximation to Swamee Jain f factor
import math
pi = math.pi
A = (pi*((D/1000)**2))/4 # m**2 Pipe area
V = (Q/1000)/A # m/s Flow velocity
print('Flow velocity = ',round(V,2),' m/s')
Re = (V*(D/1000))/v
print('Reynolds number = ',round(Re))
if Re > 4000:
    print('Reynolds number > 4000 therefore design is turbulent enough to comply with Severn Trent hydraulic standards.')
else:
        print('Reynolds number is < 4000, design not compliant with Severn Trent Standards, increase flow turbulence.')
# Swamee Jain friction factor
f = 0.25 / ((math.log10(((k/1000)/(3.71*(D/1000))+(5.74/(Re**0.9)))))**2)
# Darcy Weisbach equation
Hl = f * (L/(D/1000)) * ((V**2)/(2*g))
print('Head loss = ',"%.3f" % Hl,' m. (Swamee Jain approximation)')

Flow velocity =  5.66  m/s
Reynolds number =  848826
Reynolds number > 4000 therefore design is turbulent enough to comply with Severn Trent hydraulic standards.
Head loss =  644.826  m. (Swamee Jain approximation)


In [8]:
# Colebrook White equation
lh = 1/(f**0.5)
rh = -2*math.log10(((k/1000)/(3.7*D))+(2.51/(Re*(f**0.5))))
print('f = ',"%.5f" % f)
print('Left hand Cbw = ',"%.5f" % lh)
print('Right hand Cbw = ',"%.5f" % rh)

f =  0.02370
Left hand Cbw =  6.49503
Right hand Cbw =  9.40902


In [9]:
# Iteration to get right and left side of Colebrook White equation approximately equal
Diff = (lh - rh)**2
n = 0
while Diff > 1e-12:
    if lh > rh:
        f = f + 0.0000000001
    else:
            f = f - 0.0000000001
    lh = 1/(f**0.5)
    rh = -2*math.log10(((k/1000)/(3.7*(D/1000)))+(2.51/(Re*(f**0.5))))
    Diff = (lh - rh)**2
    n += 1
Hlcbw = f * (L/(D/1000)) * ((V**2)/(2*g))
Hldiff = ((Hl - Hlcbw)**2)**0.5
print('f = ',round(f,5))
print('LH Cbw = ',"%.5f" % lh)
print('RH Cbw = ',"%.5f" % rh)
print(f'Diff ={Diff**0.5}')
print('\n')
print('Flow velocity = ',"%.2f" % V,' m/s')
print('Head loss = ',"%.3f" % Hl,' m. (Swamee Jain approximation)')
print('Head loss = ',"%.3f" % Hlcbw,' m. (iterative method)')
print("%.3f" % Hldiff,"m difference between Colebrook White iterative method and Swamee Jain approximation")
print(f'{n} iterations to solution')

f =  0.02364
LH Cbw =  6.50398
RH Cbw =  6.50398
Diff =9.918185606139218e-07


Flow velocity =  5.66  m/s
Head loss =  644.826  m. (Swamee Jain approximation)
Head loss =  643.054  m. (iterative method)
1.772 m difference between Colebrook White iterative method and Swamee Jain approximation
651464 iterations to solution


In [10]:
# Iteration to get right and left side of Colebrook White equation approximately equal
# This method gets far faster convergence than method above

Diff = (lh - rh)
n = 0

while Diff > 1e-6 or Diff < -1e-6:
    f = (1/rh)**2 # re-arrange lh and re-calculate f by substituting rh for lh
    lh = 1/(f**0.5) # re-calculate lh with new f
    # re-calculate rh with new f
    rh = -2*math.log10(((k/1000)/(3.7*(D/1000)))+(2.51/(Re*(f**0.5))))
    Diff = (lh - rh) # re-calculate the difference
    n += 1

Hlcbw = f * (L/(D/1000)) * ((V**2)/(2*g))
Hldiff = ((Hl - Hlcbw)**2)**0.5
print('f = ',round(f,5))
print('LH Cbw = ',"%.5f" % lh)
print('RH Cbw = ',"%.5f" % rh)
print(f'Diff ={Diff}')
print('\n')
print('Flow velocity = ',"%.2f" % V,' m/s')
print('Head loss = ',"%.3f" % Hl,' m. (Swamee Jain approximation)')
print('Head loss = ',"%.3f" % Hlcbw,' m. (iterative method)')
print("%.3f" % Hldiff,"m difference between Colebrook White iterative method and Swamee Jain approximation")
print(f'{n} iterations to solution')

f =  0.02364
LH Cbw =  6.50398
RH Cbw =  6.50398
Diff =-9.918185606139218e-07


Flow velocity =  5.66  m/s
Head loss =  644.826  m. (Swamee Jain approximation)
Head loss =  643.054  m. (iterative method)
1.772 m difference between Colebrook White iterative method and Swamee Jain approximation
0 iterations to solution


In [12]:
# Input
Q = 100 # l/s
D = 150 # mm Pipe diameter
v = 1*10**-6 # m**2/s Viscosity
g = 9.81 # m/s**2
L = 2500 # m length of pipe
k = 0.3 # mm surface roughness
accuracy = 3 # accuracy of calculation in decimal places
# Prelim calcs
A = (pi*((D/1000)**2))/4 # m**2 Pipe area
V = (Q/1000)/A # m/s Flow velocity
Re = (V*(D/1000))/v
# Swamee Jain friction factor
f = 0.25 / ((math.log10(((k/1000)/(3.71*(D/1000))+(5.74/(Re**0.9)))))**2)
# Darcy Weisbach equation
def d_w_Hl(f,L,D,V,g):
    Hl = f * (L/(D/1000)) * ((V**2)/(2*g))
    return Hl
Hl = d_w_Hl(f,L,D,V,g)
print(f'Headloss using Swamee-Jain f: {Hl:.3f} m')
# Colebrook White equation
lh = 1/(f**0.5)
rh = -2*math.log10(((k/1000)/(3.7*D))+(2.51/(Re*(f**0.5))))
# recalculate friction factor using lh of Colebrook White
f_n = (1/rh)**2
# recalculate head loss using new friction factor
Hl_n = d_w_Hl(f_n,L,D,V,g)

Diff = (Hl - Hl_n)
n = 0

while Diff > 10**(-accuracy) or Diff < -10**(-accuracy):
    f = (1/rh)**2 # re-arrange lh and re-calculate f by substituting rh for lh
    lh = 1/(f**0.5) # re-calculate lh with new f
    # re-calculate rh with new f
    rh = -2*math.log10(((k/1000)/(3.7*(D/1000)))+(2.51/(Re*(f**0.5))))
    Hl_n = d_w_Hl(f,L,D,V,g)
    print(f'Headloss itteration {n+1}: {Hl_n:.3f} m')
    Diff = (Hl - Hl_n) # re-calculate the difference
    Hl = Hl_n
    n += 1
print(f'{n} itterations to solution')

Headloss using Swamee-Jain f: 644.826 m
Headloss itteration 1: 307.267 m
Headloss itteration 2: 645.677 m
Headloss itteration 3: 643.041 m
Headloss itteration 4: 643.053 m
Headloss itteration 5: 643.053 m
5 itterations to solution
